In [1]:
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from keras.utils import to_categorical

In [2]:
from tensorflow.keras.applications import VGG16

In [3]:
import os
import glob
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm
from PIL import Image, ImageOps
from tensorflow.keras import models, layers, optimizers
# from tensorflow.keras import layers
# from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [4]:
img_rows = 100
img_cols = 100

In [5]:
VGG = VGG16(weights = "imagenet",
           include_top = False,
           input_shape = (img_rows, img_cols, 3))

In [6]:
for layer in VGG.layers:
    layer.trainable = False

In [7]:
for (i,layer) in enumerate(VGG.layers):
    print(str(i) + " " + layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [8]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
#from tensorflow.keras.layers.normalization import BatchNormalization

In [9]:
def new_model(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(512, activation = "relu")(top_model)
    top_model = Dense(512, activation = "relu")(top_model)
    top_model = Dense(256, activation = "relu")(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    
    return top_model

In [10]:
model_top = new_model(VGG, 20)

In [11]:
final_model = Model(inputs = VGG.input, outputs = model_top)

In [12]:
final_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0     

In [13]:
final_model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [14]:
from keras.preprocessing.image import ImageDataGenerator

In [15]:
directory = "/Users/elias/Desktop/destination_color"

In [41]:
data_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 45,
    width_shift_range = 0.3,
    height_shift_range = 0.3,
    horizontal_flip = True,
    fill_mode = "nearest",
    validation_split = 0.2
    )

In [42]:
batch_size = 32

In [43]:
loaded_train_data = data_generator.flow_from_directory(
    directory,
    target_size = (img_rows,img_cols),
    batch_size = batch_size,
    class_mode = "categorical",
    subset='training'
    )

Found 1600 images belonging to 20 classes.


In [44]:
loaded_val_data = data_generator.flow_from_directory(
    directory,
    target_size = (img_rows,img_cols),
    batch_size = batch_size,
    class_mode = "categorical",
    subset='validation'
    )

Found 400 images belonging to 20 classes.


SyntaxError: invalid syntax (<ipython-input-20-f7d283d518a2>, line 1)

In [31]:
type(loaded_data)

tensorflow.python.keras.preprocessing.image.DirectoryIterator

In [22]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=10, verbose=1)



In [23]:
import tensorflow as tf

In [24]:
LRreducer = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor = 0.1, patience=3, verbose=1, min_lr=0)

In [25]:
EarlyStopper = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, restore_best_weights=True)

In [26]:
TensorBoard = keras.callbacks.TensorBoard(log_dir="log/{}".format(final_model), histogram_freq=0, batch_size=32, write_graph=True, write_grads=True, write_images=True, update_freq='epoch')

In [45]:
history = final_model.fit(loaded_train_data,
                    steps_per_epoch=1600/batch_size,
                    epochs=40,
                    callbacks = [LRreducer, EarlyStopper, TensorBoard],
                    validation_data = loaded_val_data,
                    validation_steps = 400/batch_size
                          )

Epoch 1/40
50/50 [==============================] - 85s 2s/step - loss: 2.7456 - accuracy: 0.1462 - val_loss: 2.5805 - val_accuracy: 0.1625
Epoch 2/40
50/50 [==============================] - 89s 2s/step - loss: 2.5120 - accuracy: 0.2125 - val_loss: 2.3360 - val_accuracy: 0.3100
Epoch 3/40
50/50 [==============================] - 91s 2s/step - loss: 2.3472 - accuracy: 0.2844 - val_loss: 2.3259 - val_accuracy: 0.2825
Epoch 4/40
50/50 [==============================] - 86s 2s/step - loss: 2.3086 - accuracy: 0.2756 - val_loss: 2.1306 - val_accuracy: 0.3525
Epoch 5/40
50/50 [==============================] - 85s 2s/step - loss: 2.1867 - accuracy: 0.3325 - val_loss: 1.9497 - val_accuracy: 0.3825
Epoch 6/40
50/50 [==============================] - 97s 2s/step - loss: 2.1109 - accuracy: 0.3487 - val_loss: 2.0084 - val_accuracy: 0.3825
Epoch 7/40
50/50 [==============================] - 100s 2s/step - loss: 2.0763 - accuracy: 0.3706 - val_loss: 1.9592 - val_accuracy: 0.4400
Epoch 8/40
50/50 [=

In [48]:
history.epoch["accuracy"]

TypeError: list indices must be integers or slices, not str